In [2]:
import pandas as pd


In [15]:
odf= pd.read_excel('db_results.xlsx',sheet_name='Results_with_units')

For converting Logic

In [44]:
import re
import numpy as np

df=odf.copy()
# Mapping superscript characters to normal digits
superscript_map = str.maketrans("⁰¹²³⁴⁵⁶⁷⁸⁹", "0123456789")

def parse_result(value):
    val_str = str(value).strip().lower()
    val_str = val_str.translate(superscript_map)  # Convert superscript numbers

    # Normalize formats like "4.7x x10(5)" → "4.7 x 10^5"
    val_str = re.sub(r"\s*x\s*x\s*", " x ", val_str)  # Fix double "x"
    val_str = re.sub(r"10\((\d+)\)", r"10^\1", val_str)  # Convert 10(5) → 10^5

    # First, check for normal multiplication (e.g., "1.88 x 1000")
    match_multiply = re.search(r"([\d\.]+)\s*[xX×*]\s*(10+)$", val_str)
    if match_multiply:
        base = float(match_multiply.group(1))
        multiplier = int(match_multiply.group(2))  # Convert to an integer
        return np.multiply(base, multiplier)  # Use NumPy for multiplication

    # Then, check for scientific notation (X x 10^Y OR X x 10Y where Y is 1-9)
    match_sci = re.search(r"([\d\.]+)\s*[xX×*]\s*10\^?(\d+)", val_str)
    if match_sci:
        base = float(match_sci.group(1))
        exponent = int(match_sci.group(2))
        return base * (10 ** exponent)

   

def convert_result(report_value):
    report_value = str(report_value).strip().replace('\n','')
    
    # Keywords mapping to 0
    if report_value.lower() in ['absent', 'negative', 'nil', 'conform/s', 'compile/s','absenl','complies','not detected','absenl','-absent','conforms','free']\
          or report_value.lower().startswith('free from') or 'it is free from' in report_value.lower() or 'absent' in report_value.lower()\
           or 'free from' in report_value.lower() or 'not observed' in report_value.lower() or 'freee from' in report_value.lower():
        return 0.0
    # Keywords mapping to 1
    elif report_value.lower() in ['present', 'positive'] or report_value.lower().startswith('present'):
        return 1.0
    # Handle BLQ 
    elif 'BL' in report_value:
        pattern = re.compile(r'(?:LOQ:?\s*|<|DL:|of\s*|\s*)(\d+\.?\d*)')
        match = pattern.search(report_value)
        if match:
            extracted_num = match.group(1)
            if '.' in extracted_num:
                num = float(extracted_num)
            else:
                num = int(extracted_num) / 10  # Assume no decimal implies thousandths
            return num / 10
        else:
            return 0.0001
          # Default for BLQ without specific value
    elif 'B.L.Q' in report_value or '(Q.L.=' in report_value:
        match = re.search(r'\(Q\.L\.*\s*=?\s*([\d\.]+)\)', report_value)
        if match:
            try:
                extracted_num = float(match.group(1))
                return extracted_num / 10  # Convert QL to result
            except ValueError:
                return None
                # Default for BLQ without specific value
        
    elif report_value.startswith('ND') or report_value.startswith('BDL'):
        # Regex to match variations like "DL: X", "DL : X", "DL X"
        pattern = re.compile(r'DL[: ]*\s*(\d+\.?\d*)')
        match = pattern.search(report_value)
        if match:
            extracted_num = float(match.group(1))
            return extracted_num / 10
    elif 'bql' in report_value.lower():
        return 0.0001
    # Handle "Less Than X" cases
    match_less_than = re.search(r'less\s*than\s*([\d\.]+)', report_value.lower())
    if match_less_than:
        extracted_num = float(match_less_than.group(1))
        return extracted_num / 10  # Convert by dividing by 10
    # Handle "<100 m" or similar cases
    match_m = re.search(r'<\s*(\d+)\s*m?', report_value)
    if match_m:
        extracted_num = float(match_m.group(1))
        return extracted_num / 10  # Convert by dividing by 10
    
    # Use parse_result function for scientific notation & multiplication cases
    parsed_value = parse_result(report_value)
    if parsed_value is not None:
        return parsed_value
    # Handle percentages
    elif '%' in report_value:
        try:
            return float(report_value.replace('%', ''))
        except ValueError:
            return None
    elif '()' in report_value:
        return float(report_value.replace('()', ''))
    elif report_value.endswith('*'):
        try:
            return float(report_value.replace('*', ''))
        except ValueError:
            return None

    # Try converting directly to float; if that fails, extract the first number
    else:
        try:
            return float(report_value)
        except ValueError:
            # If comma exists, try removing commas and converting
            if ',' in report_value and len(report_value) > 5:
                try:
                    return float(report_value.replace(',', ''))
                except ValueError:
                    pass
            # Fallback: extract the first numeric substring (e.g. "8.22" from "8.22, µg/kg")
            match = re.search(r'([-+]?\d*\.?\d+)', report_value)
            if match:
                return float(match.group(1))
            else:
                return report_value





In [45]:
df.head()

,Order ID,dsadd,commodity,state,district,sampleType,status,sampleVariant,lab,Lab Overall Compliance,...,Phosphamidon,Captafol,Ferbam,Formothion,Simazine,Diazinon,D.D.T,Fenitrothion,Fenthion,Ethyl Parathion
0,FS-AP-AN-CHILLI_POWDER-654458,Hence the product is Unsafe as per section 3(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
1,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-047566,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
2,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-780877,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
3,FS-AP-AN-CUMIN_POWDER-074499,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
4,FS-AP-AN-TURMERIC_POWDER-484432,1. The lab has tested all applicable General a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"


In [46]:
# Assuming df is already loaded
start_col = "Added Colouring Matter"
end_col = "Ethyl Parathion"

# Get the column range
col_range = df.loc[:, start_col:end_col].columns

# Apply the function to the selected columns
df[col_range] = df[col_range].applymap(convert_result)

# Display the modified DataFrame
df.head()

C:\Users\Dharya Jasuja\AppData\Local\Temp\ipykernel_18492\682680018.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[col_range] = df[col_range].applymap(convert_result)


,Order ID,dsadd,commodity,state,district,sampleType,status,sampleVariant,lab,Lab Overall Compliance,...,Phosphamidon,Captafol,Ferbam,Formothion,Simazine,Diazinon,D.D.T,Fenitrothion,Fenthion,Ethyl Parathion
0,FS-AP-AN-CHILLI_POWDER-654458,Hence the product is Unsafe as per section 3(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
1,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-047566,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
2,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-780877,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
3,FS-AP-AN-CUMIN_POWDER-074499,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
4,FS-AP-AN-TURMERIC_POWDER-484432,1. The lab has tested all applicable General a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001


In [35]:
odf

,Order ID,dsadd,commodity,state,district,sampleType,status,sampleVariant,lab,Lab Overall Compliance,...,Phosphamidon,Captafol,Ferbam,Formothion,Simazine,Diazinon,D.D.T,Fenitrothion,Fenthion,Ethyl Parathion
0,FS-AP-AN-CHILLI_POWDER-654458,Hence the product is Unsafe as per section 3(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
1,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-047566,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
2,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-780877,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
3,FS-AP-AN-CUMIN_POWDER-074499,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
4,FS-AP-AN-TURMERIC_POWDER-484432,1. The lab has tested all applicable General a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg",NaN,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,FS-WB-KO-TURMERIC_POWDER-383853,The sample conforms to the specifications laid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg"
1139,FS-WB-KO-TURMERIC_POWDER-451419,The sample is 'unsafe' by virtue of containing...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg",NaN,"BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg"
1140,FS-WB-KO-TURMERIC_WHOLE-358577,Opinion: Sample does not conforms to FSS (Labe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg","BLQ (LOQ: 0.01), mg/kg"
1141,FS-WB-SI-TURMERIC_POWDER-596351,The sample conforms to the requirements given ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,"BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg","BLQ, mg/kg"


In [47]:
df

,Order ID,dsadd,commodity,state,district,sampleType,status,sampleVariant,lab,Lab Overall Compliance,...,Phosphamidon,Captafol,Ferbam,Formothion,Simazine,Diazinon,D.D.T,Fenitrothion,Fenthion,Ethyl Parathion
0,FS-AP-AN-CHILLI_POWDER-654458,Hence the product is Unsafe as per section 3(1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
1,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-047566,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
2,FS-AP-AN-CINNAMON_(DALCHINI)_WHOLE-780877,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
3,FS-AP-AN-CUMIN_POWDER-074499,Opinion: 1. The lab has tested all applicable ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
4,FS-AP-AN-TURMERIC_POWDER-484432,1. The lab has tested all applicable General a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,NaN,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138,FS-WB-KO-TURMERIC_POWDER-383853,The sample conforms to the specifications laid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
1139,FS-WB-KO-TURMERIC_POWDER-451419,The sample is 'unsafe' by virtue of containing...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001,0.001,NaN,0.001,0.001,0.001,0.001,0.001,0.001,0.001
1140,FS-WB-KO-TURMERIC_WHOLE-358577,Opinion: Sample does not conforms to FSS (Labe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001
1141,FS-WB-SI-TURMERIC_POWDER-596351,The sample conforms to the requirements given ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001


In [48]:
df.to_excel("converted_results2.xlsx")